In [ ]:
!pip install selenium


In [ ]:
!pip install webdriver-manager

In [3]:
import random, time, bs4
from selenium import webdriver

print("Hello World")

Hello World


In [75]:
def randomTime(time):
    return time + random.random()

def instaNumber(_num):
    num = _num.replace(",", "")
    if(num[-1] == "k" or num[-1] == "K"):
        num = float(num[:-1]) * 1000
    elif(num[-1] == "m" or num[-1] == "M"):
        num = float(num[:-1]) * 1000000
    
    return int(num)

In [5]:
options = webdriver.ChromeOptions()

# headless 옵션 설정
options.add_argument('headless')
options.add_argument("no-sandbox")

# 브라우저 윈도우 사이즈
options.add_argument('window-size=1920x1080')

# 사람처럼 보이게 하는 옵션들
options.add_argument("disable-gpu")   # 가속 사용 x
options.add_argument("lang=ko_KR")    # 가짜 플러그인 탑재
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36')  # user-agent 이름 설정

In [6]:
import glob

In [33]:
driver = webdriver.Chrome(glob.glob("C:/workspace/chromedriver/*.exe")[0])

driver.get('https://www.instagram.com/')

# driver.implicitly_wait(3)

# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input').send_keys(instagram_username)
# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input').send_keys(instagram_password)

# driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button').click()

driver.implicitly_wait(3)

C:\Users\heechan\AppData\Local\Temp\ipykernel_14332\1789161145.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(glob.glob("C:/workspace/chromedriver/*.exe")[0])


In [92]:
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

actions = ActionChains(driver)

def userInfo(userId):
    result = {}
    driver.get(f'https://www.instagram.com/{userId}')

    time.sleep(3)

    postCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[1]/div/span').text
    postCount = instaNumber(postCount)
    result["postCount"] = postCount

    followerCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[2]/a/div/span').text
    followerCount = instaNumber(followerCount)
    result["followerCount"] = followerCount

    folloingCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[3]/a/div/span').text
    folloingCount = instaNumber(folloingCount)
    result["folloingCount"] = folloingCount

#     print(f"게시물 : {postCount:,}개, 팔로워 : {followerCount:,}명, 팔로잉 : {folloingCount:,}명")


    pidCount = 0
    likeCountSum = 0
    commentCountSum = 0
    
    pidRowList = driver.find_elements_by_css_selector("._ac7v._aang")

    for n, i in enumerate(pidRowList):
        pidItemList = i.find_elements_by_css_selector("._aabd._aa8k._aanf")

        for j in pidItemList:
            actions.move_to_element(j)
            actions.perform()


            li = j.find_elements_by_css_selector("li._abpm")

            likeCount = instaNumber(li[0].find_element_by_css_selector("span").text)
            commentCount = instaNumber(li[1].find_element_by_css_selector("span").text)
            
            likeCountSum += likeCount
            commentCountSum += commentCount
            
            pidCount += 1

#             print(f"종아요 : {likeCount}, 댓글 : {commentCount}")

            time.sleep(2)
    
        if(n + 1 == len(pidRowList)):
            break
    
    result["pidCount"] = pidCount
    result["likeCount"] = likeCountSum
    result["commentCount"] = commentCountSum

    # 피드마다 방문한뒤 좋아요 수랑 댓글 작성자를 가져온다.
    
    commentUserSet = set()
    driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/div[2]/article/div[1]/div/div[1]/div[1]/a').click()
    nextBtn = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[2]/div/div/div[1]/div/div[3]/div/div/div/div/div[1]/div/div/div/button')
    
    for j in range(pidCount):
        time.sleep(5)    
        html = driver.page_source
        soup = bs4.BeautifulSoup(html)
        commentList = soup.find_all("ul", {"class" : "_a9ym"})
        for i in commentList:
            commentUserSet.add(i.find("h3").text.replace("인증됨", ""))
#             print(i.find("h3").text.replace("인증됨", ""))

        nextBtn.click()
    
    result["commentUserList"] = list(commentUserSet)
    
    return result

In [94]:
userInfo("instagram")

C:\Users\heechan\AppData\Local\Temp\ipykernel_14332\2463643760.py:12: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  postCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[1]/div/span').text
C:\Users\heechan\AppData\Local\Temp\ipykernel_14332\2463643760.py:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  followerCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/section/main/div/header/section/ul/li[2]/a/div/span').text
C:\Users\heechan\AppData\Local\Temp\ipykernel_14332\2463643760.py:20: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  folloingCount = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[1]/div/div/div[1]/div[1]/sec

{'postCount': 7164,
 'followerCount': 520000000,
 'folloingCount': 130,
 'pidCount': 12,
 'likeCount': 5295000,
 'commentCount': 128967,
 'commentUserList': ['shame_dastsaz_aynur',
  'mathew_posts_shit_memes',
  'nohe_torki_ardabili',
  'nilofar.sozheh',
  'xmarty_boy_60r',
  'armin_model_gallery',
  'amitchampaneri',
  'sanakhan_0378',
  'dehkade_kalam',
  'kabirjavheri09',
  'oghenekaroitene',
  'isha____pathania',
  'topshop_cosmetic_sale',
  'ald0daniell',
  'amin.kh.16',
  'zah_ra201',
  'sahil.shaikh________10',
  'woodland_ir',
  'karuppukaruppu681',
  'lklkbi',
  'ali323_',
  'ezal_baloch',
  'maryam_mrze',
  'visha.rajput_',
  'official_malik_suhail',
  'romantic._clip_',
  'ali.kkazem1',
  'rinasonline',
  '3omi_najm',
  'royalsfff',
  'sureshpatel9141',
  'mhmd._qrb',
  '_gopal_bhadu_0029',
  '0029hanubishnoi',
  'niloofar._nahal',
  'cliip_asheghaneh78',
  'dimpilankita',
  'saat.ma3',
  '___abhirajking_22_23',
  'shaikh_raaquib_t72',
  'iamkeshavchuadhariy4',
  'memel592',